# Regularization

In statistics and machine learning, **regularization** is used to help avoid the problem of overfitting.

Models that are overfit can effectively explain observed data but generalize poorly to unseen data. 

Regularization stops our model overfitting our training data by constraining it's learned parameters, this in turn stops any variable/s having too much influence on the model.

# Formulation

We know that the least squares solution to the linear regression problem $w_{ls}$ given a set of observations $y$ and data $X$ is equal to $(X^TX)^{-1}X^Ty$. 

Least squares also has a probabilistic interpretation which allows us to understand some interesting properties about our feature vector $w$.

If we imagine our observations are drawn from a normal distribution with $\mu = Xw$ and a diagonal covariance matrix $\Sigma = \sigma^2I$ we can then solve for $w$ using maximum likelihood estimation.

$$ w_{ML} = arg\,max\;\;ln(p(y|\mu = Xw))$$

$$ w_{ML} = -\frac{1}{2\sigma^{-2}}\,\Vert y - Xw \Vert^2 - \frac{n}{2} ln(2\pi\sigma^2)$$

Least squares (LS) and maximum likelihood (ML) share the same solution:

$$ LS:   arg min\; \Vert y - Xw\Vert^2\;\Leftarrow\Rightarrow\;ML:   arg max\; -\frac{1}{2\sigma^{-2}}\,\Vert y - Xw \Vert^2 $$

In a sense we are making an independent Gaussian noise assumption about the error term $\epsilon$, this is equivalent to $$y_i = x_i^Tw + \epsilon_i, \epsilon_i \sim N(0,\sigma^2) \equiv y \sim N(Xw, \sigma^2)$$

Under the gaussian assumption $y \sim N(Xw,\sigma^2)$ it can be shown that $$ \mathbb E[w_{ml}]= w$$ and $$Var[w_{ml}]=\sigma^2(X^TX)^{-1}$$

This implies that the maximum likelihood estimate of w is unbiased, however when the value $\sigma^2(X^TX)^{-1}$ is very large then our derived $w$ is extremely sensitive to the observed $y$

One way of constraining the the values in $w$ is to apply a penalty term $\lambda g(w)$ to the least squares objective function so that it becomes: $$w = argmin \; \Vert y - Xw \Vert + \lambda g(w)$$ where $lambda > 0 :$ is a regularisation parameter and $g(w) > 0$ is a function that enforces certain desrired characters about w

# Ridge Regression

Ridge regression is one such regularization method — it uses the squared penalty on the regularisation parameter 

$$w_{rr} = argmin \; \Vert y - Xw \Vert + \lambda \Vert w \Vert^2$$.

The inclusion of $\Vert w \Vert^2$ as the regularisation parameter penalises large values in $w$.


We can solve for the ridge regression solution in exactly the same manor as we do for ordinary least squares.

$$ L = \Vert y - XW \Vert ^2 + \lambda \Vert w \Vert^2 = (y - Xw)^T(y-Xw)+\lambda w^Tw $$

To solve we take the gradient of $L$ w.r.t $w$ and set to zero

$$\nabla L = 2X^Ty+2X^TXw+2\lambda w = 0$$

it then follows that:

$$ w_{rr} = (\lambda I + X^TX)^{-1}X^Ty $$

#Code Example

Let's take a look at the House Prices dataset from Kaggle. The data contains 79 explanatory variables which describe properties of houses in Ames, Iowa as well there selling price. 

GIven the available data let's try to build a model that can predict the selling price of a house given a set of features.

In [106]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [107]:
df = pd.read_csv("housing-data.csv",index_col=0)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-

In [108]:
cols_to_drop = ["PoolQC","Fence","MiscFeature","FireplaceQu","Alley"]

df = df[[c for c in df.columns if c not in cols_to_drop]].dropna()

In [109]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [110]:
df = pd.get_dummies(df,drop_first=True)
msk = np.random.rand(len(df)) < 0.8

X_train,X_test = df.drop("SalePrice", axis=1)[msk].as_matrix(), df.drop("SalePrice", axis=1)[~msk].as_matrix()

y_train, y_test = df["SalePrice"][msk].as_matrix(), df["SalePrice"][~msk].as_matrix()

As ridge regression imposes a penalty on the size of regression coefficients, we need a way of making sure that our
coefficients are on the same scale so that the penalties are applied fairly. 

To do this we make sure our input data has mean zero and unit variance (z-scoring), we also centre our dependent variable $y$ so that it has mean zero.

In [111]:
from sklearn.preprocessing import StandardScaler
def standardize_matrix(X):
    
    X_std = StandardScaler().fit_transform(X)
    
    return X_std

X_train,X_test = standardize_matrix(X_train),standardize_matrix(X_test)

y_train = y_train - y_train.mean()

Let's create a ridge regression class. We can then use it to fit a model to our training data and in turn estimate a feature vector $w$

In [228]:
class RidgeRegression():
    
    #Fits a model to our training data and in turn produces a feature vector w
    def fit(self, X, y, l=1):
        
        #Number of features in our dataset
        p = X.shape[1]
        
        #Transpose our feature matrix X
        X_T = X.T
        
        #Identity matrix of size p
        eye = np.eye(p)
        
        C = l * eye + X.T.dot(X)
        
        # w = (lambda*I + X^tX)^-1 X^tY
        self.w = np.linalg.inv(C).dot(X.T.dot(y))
    
    #Predicts dependent variable y for a matrix with the same number of columns as the training data
    def predict(self, X):
        
        return X.dot(self.w)

r = RidgeRegression()

r.fit(X_train, y_train,l=100)

y_pred = r.predict(X_train)

np.sqrt(np.square(y_pred - y_train).sum() / len(y_train))

25346.931006886953

In [113]:
r = RidgeRegression()

r.fit(X_train, y_train,l=0.01)

y_pred = r.predict(X_train)

np.sqrt(np.square(y_pred - y_train).sum() / len(y_train))

21056.526384345994

In [114]:
from sklearn.linear_model import Ridge

clf = Ridge(alpha=0.01)

clf.fit(X_train, y_train) 

y_pred = clf.predict(X_train)

np.sqrt(np.square(y_pred - y_train).sum() / len(y_train))

21056.526384346002

In [115]:
from sklearn.linear_model import LinearRegression

clf = LinearRegression()

clf.fit(X_train, y_train) 

y_pred = clf.predict(X_train)

np.sqrt(np.square(y_pred - y_train).sum() / len(y_train))

21056.566401793225

In [226]:
from scipy.optimize import minimize

class LassoRegression():
    

    def fit(self, X, y, lam=1):
        
        p = X.shape[1]
        
        w0 = np.random.rand(p)
        
        func = lambda w,y,x,lam: ((y - x.dot(w))**2).sum() + lam*w.sum()
        
        self.w = minimize(func, w0, args=(y,X,lam))["x"]
    
    def predict(self, X):
        
        return X.dot(self.w)
        
"""    
    return ((y - X.dot(w))**2).sum() + lam*w.sum()


W = minimize(func, np.random.rand(X_train.shape[1]), args=(y_train, X_train, 100))
"""

clf = LassoRegression()
clf.fit(X_train, y_train,0.1) 

y_pred = clf.predict(X_train)

np.sqrt(np.square(y_pred - y_train).sum() / len(y_train))

21057.42025628706

In [214]:
func = lambda w,y,x: ((y - x.dot(w))**2).sum() + 1.0*w.sum()

In [215]:
func(np.array([1]),np.array([1]),np.array([1]))

1.0

In [206]:
np.sqrt(np.square(y_train - X_train.dot(W)).sum() / len(y_train))

21062.029348836368

In [142]:
w = np.random.rand(X_train.shape[1])

In [174]:
np.linalg.norm(y_train - X_train.dot(r.w),2)

755321.53245057422

In [140]:
y_train.shape

(888,)

In [141]:
X_train.shape

(888, 221)

In [173]:
(y_train - X_train.dot(r.w)).sum() + w.sum()

-3460.9763339317606

In [90]:
class RidgeRegressor(object):
    """
    Linear Least Squares Regression with Tikhonov regularization.
    More simply called Ridge Regression.
    We wish to fit our model so both the least squares residuals and L2 norm
    of the parameters are minimized.
    argmin Theta ||X*Theta - y||^2 + alpha * ||Theta||^2
    A closed form solution is available.
    Theta = (X'X + G'G)^-1 X'y
    Where X contains the independent variables, y the dependent variable and G
    is matrix alpha * I, where alpha is called the regularization parameter.
    When alpha=0 the regression is equivalent to ordinary least squares.
    http://en.wikipedia.org/wiki/Linear_least_squares_(mathematics)
    http://en.wikipedia.org/wiki/Tikhonov_regularization
    http://en.wikipedia.org/wiki/Ordinary_least_squares
    """

    def fit(self, X, y, alpha=0):
        """
        Fits our model to our training data.
        Arguments
        ----------
        X: mxn matrix of m examples with n independent variables
        y: dependent variable vector for m examples
        alpha: regularization parameter. A value of 0 will model using the
        ordinary least squares regression.
        """
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        G = alpha * np.eye(X.shape[1])
        G[0, 0] = 0  # Don't regularize bias
        self.params = np.dot(np.linalg.inv(np.dot(X.T, X) + np.dot(G.T, G)),
                             np.dot(X.T, y))

    def predict(self, X):
        """
        Predicts the dependent variable of new data using the model.
        The assumption here is that the new data is iid to the training data.
        Arguments
        ----------
        X: mxn matrix of m examples with n independent variables
        alpha: regularization parameter. Default of 0.
        Returns
        ----------
        Dependent variable vector for m examples
        """
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        return np.dot(X, self.params)

r = RidgeRegressor()

r.fit(X_train,y_train,10)

y_pred = r.predict(X_train)

np.sqrt(np.square(y_pred - y_train).sum() / len(y_train))

25730.13851952718

In [160]:
y_pred[:10]

array([ 200329.04224737,  205025.7866696 ,  201719.81815852,
        162748.1825252 ,  294229.74646662,  119512.03361856,
        284563.50232795,  132922.08983185,  114623.6370127 ,
        392210.09571446])

In [159]:
X_train.dot(w)[:10]

array([  13531.44384922,   18228.18827142,   14922.21976037,
        -24049.41587295,  107432.14806847,  -67285.56477961,
         97765.90392975,  -53875.50856632,  -72173.96138548,
        205412.49731629])

In [82]:
import numpy as np




array([[ 0.09866795, -0.24877704, -0.20127756, ..., -0.13055824,
         0.49300665, -0.34662687],
       [-0.85472276,  0.34150783, -0.06727982, ..., -0.13055824,
         0.49300665, -0.34662687],
       [ 0.09866795, -0.13072007,  0.1249778 , ..., -0.13055824,
         0.49300665, -0.34662687],
       ..., 
       [ 0.33701563, -0.20942472, -0.13229785, ..., -0.13055824,
         0.49300665, -0.34662687],
       [-0.85472276, -0.13072007, -0.053647  , ..., -0.13055824,
         0.49300665, -0.34662687],
       [-0.85472276,  0.1447462 , -0.02801265, ..., -0.13055824,
         0.49300665, -0.34662687]])

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,